In [7]:
import os
import sys

sys.path.insert(0, '../')

In [13]:
import pandas as pd
import numpy as np

from src.models.model_common import select_features

In [96]:
train_data = pd.read_csv('../data/processed/train_900000000000.csv', parse_dates=[1])

In [97]:
site_data = train_data.loc[train_data['SiteId'] == 34, :]
site_data.head()

,ForecastId,Timestamp,obs_id,SiteId,Value,Frequency,DayOfMonth,DayOfWeek,DayOfYear,DaysInMonth,...,ConsumptionPerSurfaceArea,ConsumptionPerTemperatureDiff,ConsumptionDailyMeanPerSurfaceArea,ConsumptionDailyMeanPerTemperatureDiff,ConsumptionWeeklyMeanPerSurfaceArea,ConsumptionWeeklyMeanPerTemperatureDiff,ConsumptionBiWeeklyMeanPerSurfaceArea,ConsumptionBiWeeklyMeanPerTemperatureDiff,ConsumptionMonthlyMeanPerSurfaceArea,ConsumptionMonthlyMeanPerTemperatureDiff
797228,981,2015-02-06 12:15:00,3787750,34,672.675350,9.000000e+11,6,4,37,28,...,0.360,3.383509,0.49625,4.664073,0.133073,1.250704,0.100208,0.941822,0.086917,0.816898
797229,981,2015-02-06 12:30:00,4774117,34,663.332636,9.000000e+11,6,4,37,28,...,0.355,3.384350,0.49625,4.730940,0.133073,1.268635,0.100208,0.955324,0.086917,0.828610
797230,981,2015-02-06 12:45:00,6987358,34,4690.042021,9.000000e+11,6,4,37,28,...,2.510,23.758777,0.49625,4.697328,0.133073,1.259621,0.100208,0.948537,0.086917,0.822723
797231,981,2015-02-06 13:00:00,5148673,34,625.961784,9.000000e+11,6,4,37,28,...,0.335,3.148543,0.49625,4.664073,0.133073,1.250704,0.100208,0.941822,0.086917,0.816898
797232,981,2015-02-06 13:15:00,4718014,34,616.619070,9.000000e+11,6,4,37,28,...,0.330,3.123664,0.49625,4.697328,0.133073,1.259621,0.100208,0.948537,0.086917,0.822723


In [98]:
x, y, groups = select_features(site_data, '900s')

In [99]:
from xgboost import XGBRegressor

regressor = XGBRegressor()

In [128]:
from sklearn.model_selection import GridSearchCV

gs = GridSearchCV(regressor, {
    'learning_rate': [0.11],
    'max_depth': [5],
    'n_estimators': [20],
    'colsample_bytree': [0.8],
    'colsample_bylevel': [0.8],
    'gamma': [0],
    'reg_alpha': [1.4],
    'reg_lambda': [1.2]
}, n_jobs=8, cv=8, scoring='neg_mean_squared_error')

gs.fit(x, y)

GridSearchCV(cv=8, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=8,
       param_grid={'learning_rate': [0.11], 'max_depth': [5], 'n_estimators': [20], 'colsample_bytree': [0.8], 'colsample_bylevel': [0.8], 'gamma': [0], 'reg_alpha': [1.4], 'reg_lambda': [1.2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=0)

In [129]:
gs.best_params_

{'colsample_bylevel': 0.8,
 'colsample_bytree': 0.8,
 'gamma': 0,
 'learning_rate': 0.11,
 'max_depth': 5,
 'n_estimators': 20,
 'reg_alpha': 1.4,
 'reg_lambda': 1.2}

In [95]:
gs.best_score_

-498473204.5041989